# Lesson 2: Interacting with a CSV Data

## Setup and connect to the Azure OpenAI endpoint

**Note**: The pre-configured cloud resource grants you access to the Azure OpenAI GPT model. The key and endpoint provided below are intended for teaching purposes only. Your notebook environment is already set up with the necessary keys, which may differ from those used by the instructor during the filming.

In [1]:
import os 
from dotenv import load_dotenv
import pandas as pd
from langchain_openai import AzureChatOpenAI
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent

In [2]:
load_dotenv()

True

In [3]:
model = AzureChatOpenAI(
    openai_api_version=os.getenv('OPENAI_API_VERSION'),
    azure_deployment=os.getenv('AZURE_DEPLOYMENT'),
    openai_api_key=os.getenv('OPENAI_API_KEY'),
    azure_endpoint=os.getenv('AZURE_ENDPOINT')
)

## Load the dataset

**Note**: To access the data locally, use the following code:

```
os.makedirs("data",exist_ok=True)
!wget https://covidtracking.com/data/download/all-states-history.csv -P ./data/
file_url = "./data/all-states-history.csv"
df = pd.read_csv(file_url).fillna(value = 0)
```

In [4]:
df = pd.read_csv("./data/all-states-history.csv").fillna(value = 0)

## Prepare the Langchain dataframe agent

In [5]:
agent = create_pandas_dataframe_agent(llm=model,df=df,verbose=True,allow_dangerous_code=True)

agent.invoke("how many rows are there?")



> Entering new AgentExecutor chain...
To find out how many rows are in the DataFrame `df`, you can use the `len()` function, which returns the number of rows in a DataFrame. Let's execute this command.

Thought: I will use the `len()` function to get the number of rows in the DataFrame.
Action: python_repl_ast
Action Input: len(df)20780I now know the final answer.
Final Answer: There are 20,780 rows in the DataFrame `df`.

> Finished chain.


{'input': 'how many rows are there?',
 'output': 'There are 20,780 rows in the DataFrame `df`.'}

## Design your prompt and ask your question

In [6]:
CSV_PROMPT_PREFIX = """
First set the pandas display options to show all the columns,
get the column names, then answer the question.
"""

CSV_PROMPT_SUFFIX = """
- **ALWAYS** before giving the Final Answer, try another method.
Then reflect on the answers of the two methods you did and ask yourself
if it answers correctly the original question.
If you are not sure, try another method.
- If the methods tried do not give the same result,reflect and
try again until you have two methods that have the same result.
- If you still cannot arrive to a consistent result, say that
you are not sure of the answer.
- If you are sure of the correct answer, create a beautiful
and thorough response using Markdown.
- **DO NOT MAKE UP AN ANSWER OR USE PRIOR KNOWLEDGE,
ONLY USE THE RESULTS OF THE CALCULATIONS YOU HAVE DONE**.
- **ALWAYS**, as part of your "Final Answer", explain how you got
to the answer on a section that starts with: "\n\nExplanation:\n".
In the explanation, mention the column names that you used to get
to the final answer.
"""

QUESTION = "How may patients were hospitalized during July 2021" 
"in Texas, and nationwide as the total of all states?"
"Use the hospitalizedIncrease column" 


agent.invoke(CSV_PROMPT_PREFIX + QUESTION + CSV_PROMPT_SUFFIX)



> Entering new AgentExecutor chain...
Sure, let's proceed step by step. First, I'll set the pandas display options to show all the columns and then retrieve the column names to understand the dataset better. After that, I'll calculate the number of patients hospitalized during July 2021 using two different methods.

### Step 1: Set display options and get column names

```python
import pandas as pd

# Set pandas display options to show all columns
pd.set_option('display.max_columns', None)

# Display column names
df.columns
```

### Step 2: Calculate the number of patients hospitalized during July 2021

#### Method 1: Using date filtering and summing the `hospitalizedIncrease` column

```python
# Convert date column to datetime
df['date'] = pd.to_datetime(df['date'])

# Filter for July 2021 and sum the hospitalizedIncrease for that period
july_hospitalized = df[(df['date'].dt.month == 7) & (df['date'].dt.year == 2021) & (df['state'] == 'AL')]['hospitalizedIncrease'].sum()
july_hospit

ValueError: An output parsing error occurred. In order to pass this error back to the agent and have it try again, pass `handle_parsing_errors=True` to the AgentExecutor. This is the error: Could not parse LLM output: `It seems like there was an error in the execution due to an unterminated string literal. I'll correct this and proceed with executing the steps to find out the number of patients hospitalized in Alabama during July 2021 using two different methods.

Let's execute the corrected code:

### Step 1: Set display options and get column names

```python
import pandas as pd

# Set pandas display options to show all columns
pd.set_option('display.max_columns', None)

# Display column names
df.columns
```

### Step 2: Calculate the number of patients hospitalized during July 2021

#### Method 1: Using date filtering and summing the `hospitalizedIncrease` column

```python
# Convert date column to datetime
df['date'] = pd.to_datetime(df['date'])

# Filter for July 2021 and sum the 